In [0]:
from pyspark.sql.functions import *

In [0]:
%sql
USE CATALOG melpark_azure;

CREATE SCHEMA IF NOT EXISTS silver;

In [0]:
base_uri='abfss://bronze@melpark.dfs.core.windows.net'

# Parking Sensors Stream

In [0]:
# Configuration
source_table = "melpark_azure.bronze.parking_sensors"
target_table = "melpark_azure.silver.parking_sensors"
checkpoint_path = f"{base_uri}/_checkpoints/parking_sensors_silver"

In [0]:
location_schema = "lat DOUBLE, lon DOUBLE"

print(f"Streaming from {source_table} -> {target_table}")

Streaming from melpark_azure.bronze.parking_sensors -> melpark_azure.silver.parking_sensors


In [0]:
# 1. Read the Bronze Stream
df_bronze = spark.readStream.table(source_table)

In [0]:
# 2. Transform
df_silver = df_bronze.select(
    col("after.kerbsideid").alias("bay_id"),          
    col("after.status_description").alias("status"),  
    col("after.zone_number"),                          
    # Parse location (which is inside 'after')
    from_json(col("after.location"), location_schema).alias("parsed_location"),
    # Convert timestamp (inside 'after')
    col("after.status_timestamp").cast("timestamp").alias("event_time")
).select(
    "bay_id", 
    "status", 
    "zone_number", 
    "parsed_location.lat", 
    "parsed_location.lon", 
    "event_time"
)

In [0]:
# 3. Write to Silver (Append Mode)
query = (df_silver.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path)
    .option("mergeSchema", "true")
    .table(target_table)
)

# Parking Bays


In [0]:
source_table = "melpark_azure.bronze.parking_bays"
target_table = "melpark_azure.silver.parking_bays"
df_bays_bronze = spark.table(source_table)

In [0]:
print("Previewing Bronze Data...")
display(df_bays_bronze.limit(10))

Previewing Bronze Data...


kerbsideid,lastupdated,latitude,location,longitude,roadsegmentdescription,roadsegmentid,_ingested_at,ingested_at
null,2024-08-08,-37.8029803,"{""lat"":-37.8029803,""lon"":144.9540432}",144.9540432,Howard Street between Queensberry Street and Courtney Street,21105,2026-01-16T04:20:52.07026Z,2026-01-16T06:56:57.772689Z
null,2024-08-08,-37.8235593,"{""lat"":-37.8235593,""lon"":144.9797604}",144.9797604,Olympic Boulevard between Punt Road and Batman Avenue,22774,2026-01-16T04:20:52.07026Z,2026-01-16T06:56:57.772689Z
null,2023-10-31,-37.8102165,"{""lat"":-37.8102165,""lon"":144.9825395}",144.9825395,Albert Street between Clarendon Street and Eades Street,20898,2026-01-16T04:20:52.07026Z,2026-01-16T06:56:57.772689Z
11144,2023-10-31,-37.8082679,"{""lat"":-37.8082679,""lon"":144.9547044}",144.9547044,William Street between Dudley Street and Walsh Street,21404,2026-01-16T04:20:52.07026Z,2026-01-16T06:56:57.772689Z
null,2023-10-31,-37.7939919,"{""lat"":-37.7939919,""lon"":144.972174}",144.972174,Neill Street between Rathdowne Street and Canning Street,22788,2026-01-16T04:20:52.07026Z,2026-01-16T06:56:57.772689Z
null,2024-11-19,-37.8099691,"{""lat"":-37.8099691,""lon"":144.9627693}",144.9627693,La Trobe Street between Swanston Street and Elizabeth Street,20022,2026-01-16T04:20:52.07026Z,2026-01-16T06:56:57.772689Z
null,2023-10-31,-37.8090056,"{""lat"":-37.8090056,""lon"":144.94475}",144.94475,Hawke Street between Railway Place and Adderley Street,21470,2026-01-16T04:20:52.07026Z,2026-01-16T06:56:57.772689Z
null,2023-10-31,-37.8250277,"{""lat"":-37.8250277,""lon"":144.9456445}",144.9456445,Lorimer Street between Westgate Freeway and Hartley Street,22126,2026-01-16T04:20:52.07026Z,2026-01-16T06:56:57.772689Z
null,2024-08-08,-37.8274301,"{""lat"":-37.8274301,""lon"":144.9241567}",144.9241567,Vegemite Way between Salmon Street and Douglas Street,22150,2026-01-16T04:20:52.07026Z,2026-01-16T06:56:57.772689Z
null,2023-10-31,-37.8366932,"{""lat"":-37.8366932,""lon"":144.980811}",144.980811,Mason Street between Park Street and Randall Place,22252,2026-01-16T04:20:52.07026Z,2026-01-16T06:56:57.772689Z


In [0]:
df_bays_silver = df_bays_bronze.select(
    # Rename 'kerbsideid' to 'bay_id' and ensure it's an Integer
    col("kerbsideid").cast("int").alias("bay_id"),
    
    # Cast lat/lon to Double (numbers)
    col("latitude").cast("double"),
    col("longitude").cast("double"),
    
    # Keep the description
    col("roadsegmentdescription").alias("description"),
    
    # Cast the update time
    col("lastupdated").cast("timestamp").alias("last_updated")
).filter(
    # CRITICAL: We only keep bays that actually have an ID (Sensor Bays)
    col("bay_id").isNotNull()
)

In [0]:
# 3. Validation: Check if we have any duplicates created
duplicate_check = df_bays_silver.groupBy("bay_id").count().filter("count > 1")

In [0]:
if duplicate_check.count() == 0:
    print("Data Validity Check: No duplicate Bay IDs found.")
else:
    print("Warning: Duplicate Bay IDs detected!")

In [0]:
# 1. Find a Bay ID that appears more than once
dup_id = df_bays_silver.groupBy("bay_id").count().filter("count > 1").first()["bay_id"]

print(f"Inspecting duplicates for Bay ID: {dup_id}")

# 2. Show all rows for that ID to see the difference
display(df_bays_silver.filter(f"bay_id = {dup_id}"))

Inspecting duplicates for Bay ID: 17244


bay_id,latitude,longitude,description,last_updated
17244,-37.8038942,144.9593095,Berkeley Street between Queensberry Street and Pelham Street,2024-11-14T00:00:00Z
17244,-37.8160807,144.9758631,Wellington Parade South between Wellington Crescent and Flinders Street,2023-10-31T00:00:00Z


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, desc

# 1. Define the Window: Group by ID, Sort by Date (Newest first)
window_spec = Window.partitionBy("bay_id").orderBy(desc("last_updated"))

# 2. Rank & Filter
df_bays_deduped = df_bays_silver.withColumn("rank", row_number().over(window_spec)) \
    .filter("rank == 1") \
    .drop("rank")

print("Duplicates removed.")

Duplicates removed.


In [0]:
df_bays_deduped.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(target_table)

print("Success! Silver Parking Bays table is live.")

Success! Silver Parking Bays table is live.


In [0]:
%sql
SELECT count(*) as total_rows, count(distinct bay_id) as unique_ids 
FROM melpark_azure.silver.parking_bays;

total_rows,unique_ids
5324,5324


# Parking Meters

In [0]:
source_table = "melpark_azure.bronze.parking_meters"
target_table = "melpark_azure.silver.parking_meters"

In [0]:
df_meters_bronze = spark.table(source_table)

print("Previewing Parking Meters Data...")
display(df_meters_bronze.limit(10))

Previewing Parking Meters Data...


asset_id,barcode,creditcard,latitude,location,location_description,longitude,meter_id,tapandgo,_ingested_at,ingested_at
1647592,MPM1647592,Yes,-37.8196338,"{""lat"":-37.8196338,""lon"":144.9448746}","Bourke Street between Harbour Esplanade and Enterprize Way, approximately 11m North of Toss'D, 7 Buckley Walk, DOCKLANDS VIC 3008",144.9448746,945A,Yes,2026-01-16T04:20:47.086473Z,2026-01-16T06:57:06.754468Z
1805666,MPM1805666,null,-37.7916518,"{""lat"":-37.7916518,""lon"":144.9453832}","Flemington Road between Melrose Street and Dryburgh Street, approximately 54m NE of 295 Flemington Road, NORTH MELBOURNE VIC 3051",144.9453832,149C,Yes,2026-01-16T04:20:47.086473Z,2026-01-16T06:57:06.754468Z
1851931,MPM1851931,Yes,-37.8130558,"{""lat"":-37.8130558,""lon"":144.9829596}","Clarendon Street between Hotham Street and Gipps Street, approximately 35m SW of Bishopscourt, 84-122 Clarendon Street, EAST MELBOURNE VIC 3002",144.9829596,875B,Yes,2026-01-16T04:20:47.086473Z,2026-01-16T06:57:06.754468Z
1851936,MPM1851936,Yes,-37.8033287,"{""lat"":-37.8033287,""lon"":144.9562282}","Peel Street between Queensberry Street and Elizabeth Street, approximately 5m SE of Media Arts Lawyers Pty Ltd, 187-189 Peel Street, NORTH MELBOURNE VIC 3051",144.9562282,231C,Yes,2026-01-16T04:20:47.086473Z,2026-01-16T06:57:06.754468Z
1647618,MPM1647618,Yes,-37.8288337,"{""lat"":-37.8288337,""lon"":144.9702821}","Coventry Street between St Kilda Road and Wells Street, approximately 27m NW of Wilson Parking, Basement 312 St Kilda Road, SOUTHBANK VIC 3006",144.9702821,788D,Yes,2026-01-16T04:20:47.086473Z,2026-01-16T06:57:06.754468Z
1802357,MPM1802357,Yes,-37.7783095,"{""lat"":-37.7783095,""lon"":144.9614764}","Park Street between Royal Parade and Bowen Crescent, approximately 82m NW of Princes Park Carlton Bowls Club Inc, Princes Park Carlton Bowls Club, Princes Park, 109 Bowen Crescent, CARLTON NORTH VIC 3054",144.9614764,483B,Yes,2026-01-16T04:20:47.086473Z,2026-01-16T06:57:06.754468Z
1647114,MPM1647114,No,-37.8449573,"{""lat"":-37.8449573,""lon"":144.9803257}","Commercial Road between St Kilda Road and Punt Road, approximately 23m East of Parkview Dental, Suite 1, 20 Commercial Road, MELBOURNE VIC 3004",144.9803257,813E,Yes,2026-01-16T04:20:47.086473Z,2026-01-16T06:57:06.754468Z
1647178,MPM1647178,Yes,-37.8266846,"{""lat"":-37.8266846,""lon"":144.9690526}",Wells Street between Grant Street and Miles Street,144.9690526,765D,Yes,2026-01-16T04:20:47.086473Z,2026-01-16T06:57:06.754468Z
1805858,MPM1805858,null,-37.8269482,"{""lat"":-37.8269482,""lon"":144.980142}","Alexandra Avenue between Swan Street Bridge and Anderson Street, approximately 72m NE of Plant Craft Cottage, Royal Botanic Gardens Victoria, 1A Alexandra Avenue, MELBOURNE VIC 3004",144.980142,832B,Yes,2026-01-16T04:20:47.086473Z,2026-01-16T06:57:06.754468Z
1647201,MPM1647201,Yes,-37.8187687,"{""lat"":-37.8187687,""lon"":144.9498896}","Aurora Lane between Bourke Street and Collins Street, approximately 40m NW of Travelodge Hotel Melbourne Docklands, Travelodge Docklands, 66 Aurora Lane, DOCKLANDS VIC 3008",144.9498896,901A,Yes,2026-01-16T04:20:47.086473Z,2026-01-16T06:57:06.754468Z


In [0]:
df_meters_clean = df_meters_bronze.select(
    col("meter_id"),
    col("asset_id").cast("int"),
    # Convert "Yes"/"No" to Boolean (True/False). If null, we assume False (safe default).
    when(col("creditcard") == "Yes", True).otherwise(False).alias("has_credit_card"),
    when(col("tapandgo") == "Yes", True).otherwise(False).alias("has_tap_and_go"),
    col("latitude").cast("double"),
    col("longitude").cast("double"),
    col("location_description").alias("description"),
    # Keep ingestion time for deduplication
    col("ingested_at")
).filter(col("meter_id").isNotNull())

In [0]:
# Since we don't have 'last_updated', we trust the most recently ingested file.
window_spec = Window.partitionBy("meter_id").orderBy(desc("ingested_at"))

df_meters_deduped = df_meters_clean.withColumn("rank", row_number().over(window_spec)) \
    .filter("rank == 1") \
    .drop("rank", "ingested_at")

In [0]:
df_meters_deduped.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(target_table)

print("Success! Silver Parking Meters table created.")

Success! Silver Parking Meters table created.


In [0]:
%sql
SELECT * FROM melpark_azure.silver.parking_meters LIMIT 10;

meter_id,asset_id,has_credit_card,has_tap_and_go,latitude,longitude,description
108A,1805900,false,true,-37.7765084,144.9464781,"Park Street between Oak Street and West Coburg Tramway, approximately 59m East of Parkville Youth Justice Precinct, 900 Park Street, PARKVILLE VIC 3052"
108B,1851939,true,true,-37.7766131,144.9472863,Park Street between Oak Street and West Coburg Tramway
109A,1851940,true,true,-37.7771264,144.9516542,Park Street between West Coburg Tramway and Royal Parade
109B,1805868,false,true,-37.7773412,144.9534139,"Park Street between West Coburg Tramway and Royal Parade, approximately 81m West of Royal Park Brunswick Cricket Club Inc, Western Pavilion, Royal Park, 776 Park Street, PARKVILLE VIC 3052"
110A,1805869,false,true,-37.7777467,144.9567453,"Park Street between West Coburg Tramway and Royal Parade, approximately 57m East of City of Melbourne, Royal Park Depot, 750 Park Street, PARKVILLE VIC 3052"
110B,1851941,true,true,-37.7780058,144.9589011,"Park Street between West Coburg Tramway and Royal Parade, approximately 53m NW of Royal Park Lodge, 369 The Avenue, PARKVILLE VIC 3052"
117A,1647773,true,true,-37.8107859,144.9460187,"Rosslyn Street between Railway Place and Adderley Street, approximately 12m SW of Big Door Studios, 280-286 Rosslyn Street, WEST MELBOURNE VIC 3003"
117B,1784229,true,true,-37.8106931,144.9467358,"Rosslyn Street between Railway Place and Adderley Street, approximately 27m SE of Ebike100 Pty Ltd, 260-262 Rosslyn Street, WEST MELBOURNE VIC 3003"
117C,1805628,false,true,-37.8104353,144.9467363,"Rosslyn Street between Railway Place and Adderley Street, approximately 3m SE of 252-258 Rosslyn Street, WEST MELBOURNE VIC 3003"
117D,1805625,false,true,-37.8109249,144.9462634,"Rosslyn Street between Railway Place and Adderley Street, approximately 27m SE of Big Door Studios, 280-286 Rosslyn Street, WEST MELBOURNE VIC 3003"


# Parking Zones Plates

In [0]:
source_table = "melpark_azure.bronze.parking_zones_plates"
target_table = "melpark_azure.silver.parking_zones_plates"

In [0]:
df_zones_bronze = spark.table(source_table)

print("Previewing Parking Zones Data...")
display(df_zones_bronze.limit(10))

Previewing Parking Zones Data...


parkingzone,restriction_days,restriction_display,time_restrictions_finish,time_restrictions_start,_ingested_at,ingested_at
7854,Sat-Sun,MP3P,22:00:00,07:00:00,2026-01-16T04:20:47.086698Z,2026-01-16T06:58:12.792938Z
7567,Mon-Fri,MP2P,19:00:00,16:00:00,2026-01-16T04:20:47.086698Z,2026-01-16T06:58:12.792938Z
7104,Sat,1P,12:30:00,07:30:00,2026-01-16T04:20:47.086698Z,2026-01-16T06:58:12.792938Z
7528,Mon-Fri,LZ30,16:00:00,07:00:00,2026-01-16T04:20:47.086698Z,2026-01-16T06:58:12.792938Z
7436,Mon-Fri,MP2P,19:00:00,07:00:00,2026-01-16T04:20:47.086698Z,2026-01-16T06:58:12.792938Z
7774,Mon-Fri,MP2P,22:00:00,19:00:00,2026-01-16T04:20:47.086698Z,2026-01-16T06:58:12.792938Z
7449,Sun,FP2P,18:30:00,07:30:00,2026-01-16T04:20:47.086698Z,2026-01-16T06:58:12.792938Z
7520,Mon-Fri,1P,18:30:00,07:30:00,2026-01-16T04:20:47.086698Z,2026-01-16T06:58:12.792938Z
7383,Mon-Fri,MP2P,22:00:00,19:00:00,2026-01-16T04:20:47.086698Z,2026-01-16T06:58:12.792938Z
7553,Mon-Fri,LZ30,16:00:00,10:00:00,2026-01-16T04:20:47.086698Z,2026-01-16T06:58:12.792938Z


In [0]:
df_zones_clean = df_zones_bronze.select(
    col("parkingzone").cast("int").alias("zone_number"),
    col("restriction_days"),
    col("restriction_display"),
    col("time_restrictions_start").alias("start_time"),
    col("time_restrictions_finish").alias("end_time"),
    col("ingested_at")
).filter(col("zone_number").isNotNull())

In [0]:
window_spec = Window.partitionBy("zone_number", "restriction_days", "start_time") \
    .orderBy(desc("ingested_at"))

df_zones_deduped = df_zones_clean.withColumn("rank", row_number().over(window_spec)) \
    .filter("rank == 1") \
    .drop("rank", "ingested_at")

In [0]:
df_zones_deduped.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(target_table)

print("Success! Silver Parking Zones table created.")

Success! Silver Parking Zones table created.


In [0]:
%sql
SELECT * FROM melpark_azure.silver.parking_zones_plates LIMIT 10;

zone_number,restriction_days,restriction_display,start_time,end_time
7000,Mon-Sun,5P,06:00:00,17:00:00
7001,Mon-Fri,2P,07:30:00,18:30:00
7001,Sat,2P,07:30:00,12:30:00
7002,Mon-Fri,4P,07:30:00,18:30:00
7002,Sat,4P,07:30:00,12:30:00
7003,Mon-Fri,2P,07:30:00,18:30:00
7003,Sat,2P,07:30:00,12:30:00
7004,Mon-Fri,2P,07:30:00,18:30:00
7004,Sat,2P,07:30:00,12:30:00
7007,Mon-Fri,2P,07:30:00,18:30:00


# Parking restrictions

In [0]:
source_table = "melpark_azure.bronze.parking_restrictions"
target_table = "melpark_azure.silver.parking_restrictions"

In [0]:
df_restrictions_bronze = spark.table(source_table)

print("Previewing Parking Restrictions Data...")
display(df_restrictions_bronze.limit(10))

Previewing Parking Restrictions Data...


bayid,description1,description2,description3,description4,description5,description6,deviceid,disabilityext1,disabilityext2,disabilityext3,disabilityext4,disabilityext5,disabilityext6,duration1,duration2,duration3,duration4,duration5,duration6,effectiveonph1,effectiveonph2,effectiveonph3,effectiveonph4,effectiveonph5,effectiveonph6,endtime1,endtime2,endtime3,endtime4,endtime5,endtime6,exemption1,exemption2,exemption3,exemption4,exemption5,exemption6,fromday1,fromday2,fromday3,fromday4,fromday5,fromday6,starttime1,starttime2,starttime3,starttime4,starttime5,starttime6,today1,today2,today3,today4,today5,today6,typedesc1,typedesc2,typedesc3,typedesc4,typedesc5,typedesc6,_ingested_at,ingested_at
3894,2P MTR M-SAT 7:30-18:30,2P SUN 7:30-18:30,null,null,null,null,30649,240,240,null,null,null,null,120,120,null,null,null,null,0,0,null,null,null,null,0001-01-01T18:30:00+00:00,0001-01-01T18:30:00+00:00,null,null,null,null,null,null,null,null,null,null,1,0,null,null,null,null,0001-01-01T07:30:00+00:00,0001-01-01T07:30:00+00:00,null,null,null,null,6,0,null,null,null,null,2P Meter,2P,null,null,null,null,2026-01-16T04:20:50.038843Z,2026-01-16T06:58:09.469699Z
5601,2P MTR M-F 9:30-18:30,2P MTR SAT 7:30-12:30,null,null,null,null,30420,240,240,null,null,null,null,120,120,null,null,null,null,0,0,null,null,null,null,0001-01-01T18:30:00+00:00,0001-01-01T12:30:00+00:00,null,null,null,null,null,null,null,null,null,null,1,6,null,null,null,null,0001-01-01T09:30:00+00:00,0001-01-01T07:30:00+00:00,null,null,null,null,5,6,null,null,null,null,2P Meter,2P Meter,null,null,null,null,2026-01-16T04:20:50.038843Z,2026-01-16T06:58:09.469699Z
8244,P5 M-SUN 7:30-18:30,1P M-SUN 6:30PM-11PM,null,null,null,null,25623,10,120,null,null,null,null,5,60,null,null,null,null,1,1,null,null,null,null,0001-01-01T18:30:00+00:00,0001-01-01T23:00:00+00:00,null,null,null,null,null,null,null,null,null,null,1,1,null,null,null,null,0001-01-01T07:30:00+00:00,0001-01-01T18:30:00+00:00,null,null,null,null,0,0,null,null,null,null,P 05 Min,1P,null,null,null,null,2026-01-16T04:20:50.038843Z,2026-01-16T06:58:09.469699Z
4282,2P TKT A M-SAT 7:30-20:30,2P SUN 7:30-18:30,null,null,null,null,24052,240,240,null,null,null,null,120,120,null,null,null,null,0,0,null,null,null,null,0001-01-01T20:30:00+00:00,0001-01-01T18:30:00+00:00,null,null,null,null,null,null,null,null,null,null,1,0,null,null,null,null,0001-01-01T07:30:00+00:00,0001-01-01T07:30:00+00:00,null,null,null,null,6,0,null,null,null,null,2P Ticket A,2P,null,null,null,null,2026-01-16T04:20:50.038843Z,2026-01-16T06:58:09.469699Z
8671,2P RPA 1B 7:30-23:00 M-SUN,null,null,null,null,null,27914,240,null,null,null,null,null,120,null,null,null,null,null,1,null,null,null,null,null,0001-01-01T23:00:00+00:00,null,null,null,null,null,Resident Permit Area 1B,null,null,null,null,null,1,null,null,null,null,null,0001-01-01T07:30:00+00:00,null,null,null,null,null,0,null,null,null,null,null,2P,null,null,null,null,null,2026-01-16T04:20:50.038843Z,2026-01-16T06:58:09.469699Z
8877,2P M-SAT 7:30-23:00,null,null,null,null,null,29718,240,null,null,null,null,null,120,null,null,null,null,null,0,null,null,null,null,null,0001-01-01T23:00:00+00:00,null,null,null,null,null,Resident Permit Area 5,null,null,null,null,null,1,null,null,null,null,null,0001-01-01T07:30:00+00:00,null,null,null,null,null,6,null,null,null,null,null,2P,null,null,null,null,null,2026-01-16T04:20:50.038843Z,2026-01-16T06:58:09.469699Z
5159,2P MTR M-SAT 7:30-20:30,1P SUN 7:30-18:30,null,null,null,null,26172,240,120,null,null,null,null,120,60,null,null,null,null,0,0,null,null,null,null,0001-01-01T20:30:00+00:00,0001-01-01T18:30:00+00:00,null,null,null,null,null,null,null,null,null,null,1,0,null,null,null,null,0001-01-01T07:30:00+00:00,0001-01-01T07:30:00+00:00,null,null,null,null,6,0,null,null,null,null,2P Meter,1P,null,null,null,null,2026-01-16T04:20:50.038843Z,2026-01-16T06:58:09.469699Z
6320,1/4P M-F 7:30-9:30,2P MTR M-F 9:30-16:00,1/4P M-F 16:00-18:30,2P S-S 7:30-18:30,null,null,28599,30,240,30,2

In [0]:
# 2. Define the Stack Expression
# We want to turn description1..6, duration1..6, etc., into single columns.
# The format is: stack(6, '1', desc1, dur1..., '2', desc2, dur2...)
stack_expr = """
    stack(6,
        1, description1, duration1, starttime1, endtime1, fromday1, effectiveonph1, typedesc1,
        2, description2, duration2, starttime2, endtime2, fromday2, effectiveonph2, typedesc2,
        3, description3, duration3, starttime3, endtime3, fromday3, effectiveonph3, typedesc3,
        4, description4, duration4, starttime4, endtime4, fromday4, effectiveonph4, typedesc4,
        5, description5, duration5, starttime5, endtime5, fromday5, effectiveonph5, typedesc5,
        6, description6, duration6, starttime6, endtime6, fromday6, effectiveonph6, typedesc6
    ) as (rule_number, description, duration, start_time, end_time, day_range, on_public_hols, type_desc)
"""

In [0]:
df_restr_silver = df_restrictions_bronze.select(
    col("bayid").cast("int").alias("bay_id"),
    # Inject our stack command here
    expr(stack_expr),
    col("ingested_at")
).filter(
    # Remove the empty slots (e.g., if a bay only has 2 rules, drop 3,4,5,6)
    col("description").isNotNull()
)

In [0]:
df_restr_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(target_table)

print("Success! Normalized Parking Restrictions table created.")

Success! Normalized Parking Restrictions table created.
